<a href="https://colab.research.google.com/github/bochendong/diffusion-model/blob/main/DDPM/DDPM_DANN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from contextlib import contextmanager
from copy import deepcopy
import math
import os
import glob
from PIL import Image
from IPython import display
from matplotlib import pyplot as plt
import torch
from torch import optim, nn
from torch.nn import functional as F
from torch.autograd import Function
from torch.utils import data
from torchvision import datasets, transforms, utils
from torch.utils.data import DataLoader, Dataset, TensorDataset
from torchvision.transforms import functional as TF
from tqdm.notebook import tqdm, trange
from torchvision.models import resnet50
from torchvision.utils import save_image

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

print(device)

cpu


In [3]:
def get_alphas_sigmas(t):
    """
    Returns the scaling factors for the clean image (alpha) and for the
    noise (sigma), given a timestep.
    """
    return torch.cos(t * math.pi / 2), torch.sin(t * math.pi / 2)

if (os.path.exists("./output")) == False:
    os.mkdir("output")

files = glob.glob("./output/*.png")

for f in files:
    os.remove(f)

# Training Set

In [4]:
batch_size = 64
epoches = 30
ema_decay = 0.999
steps = 500
eta = 1.

guidance_scale = 2.

tf = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

In [5]:
source_set = datasets.CIFAR10(root='data', train=True, download=True, transform=tf, target_transform=lambda x: x if x < 5 else -1)
source_dl = DataLoader(source_set, batch_size=batch_size, shuffle=True, num_workers=4, pin_memory=True)

target_set = datasets.CIFAR10(root='data', train=True, download=True, transform=tf, target_transform=lambda x: x if (x <= 7 and x >= 5) else -1)
target_dl = DataLoader(target_set, batch_size=batch_size, shuffle=True, num_workers=4, pin_memory=True)

test_set = datasets.CIFAR10(root='data', train=True, download=True, transform=tf, target_transform=lambda x: x if (x >= 8) else -1)
test_dl = DataLoader(test_set, batch_size=batch_size, shuffle=True, num_workers=4, pin_memory=True)

Files already downloaded and verified


/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:558: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


Files already downloaded and verified
Files already downloaded and verified


# Model

### ResidualBlock

In [6]:
class ResidualBlock(nn.Module):
    def __init__(self, main, skip=None):
        super().__init__()
        self.main = nn.Sequential(*main)
        self.skip = skip if skip else nn.Identity()

    def forward(self, input):
        return self.main(input) + self.skip(input)

class ResConvBlock(ResidualBlock):
    def __init__(self, c_in, c_mid, c_out, is_last=False):
        skip = None if c_in == c_out else nn.Conv2d(c_in, c_out, 1, bias=False)
        super().__init__([
            nn.Conv2d(c_in, c_mid, 3, padding=1),
            nn.Dropout2d(0.1, inplace=True),
            nn.ReLU(inplace=True),
            nn.Conv2d(c_mid, c_out, 3, padding=1),
            nn.Dropout2d(0.1, inplace=True) if not is_last else nn.Identity(),
            nn.ReLU(inplace=True) if not is_last else nn.Identity(),
        ], skip)

### Self Attention

In [7]:
class SelfAttention2d(nn.Module):
    def __init__(self, c_in, n_head=1, dropout_rate=0.1):
        super().__init__()
        assert c_in % n_head == 0
        self.norm = nn.GroupNorm(1, c_in)
        self.n_head = n_head
        self.qkv_proj = nn.Conv2d(c_in, c_in * 3, 1)
        self.out_proj = nn.Conv2d(c_in, c_in, 1)
        self.dropout = nn.Dropout2d(dropout_rate, inplace=True)

    def forward(self, input):
        n, c, h, w = input.shape
        qkv = self.qkv_proj(self.norm(input))
        qkv = qkv.view([n, self.n_head * 3, c // self.n_head, h * w]).transpose(2, 3)
        q, k, v = qkv.chunk(3, dim=1)
        scale = k.shape[3]**-0.25
        att = ((q * scale) @ (k.transpose(2, 3) * scale)).softmax(3)
        y = (att @ v).transpose(2, 3).contiguous().view([n, c, h, w])
        return input + self.dropout(self.out_proj(y))

### FourierFeatures

In [8]:
class FourierFeatures(nn.Module):
    def __init__(self, in_features, out_features, std=1.):
        super().__init__()
        assert out_features % 2 == 0
        self.weight = nn.Parameter(torch.randn([out_features // 2, in_features]) * std)

    def forward(self, input):
        f = 2 * math.pi * input @ self.weight.T
        return torch.cat([f.cos(), f.sin()], dim=-1)

### Feature Embedding

In [9]:
class FeatureEmbedding(nn.Module):
    def __init__(self):
        super(FeatureEmbedding, self).__init__()
        self.fc1 = nn.Linear(1000, 2048)
        self.fc2 = nn.Linear(2048, 3072)

        self.pretrained_model = resnet50(pretrained=True)
        self.pretrained_model = self.pretrained_model.to(device)

    def forward(self, x):
        with torch.no_grad():
            x = self.pretrained_model(x)
        x = self.fc1(x)
        x = self.fc2(x)
        x = x.view(-1, 3, 32, 32)
        return x

# Domain Classifier

In [10]:
class ReverseLayerF(Function):
    @staticmethod
    def forward(ctx, x, alpha):
        ctx.alpha = alpha

        return x.view_as(x)

    @staticmethod
    def backward(ctx, grad_output):
        output = grad_output.neg() * ctx.alpha

        return output, None

class DomainClassifier(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(256 * 8 * 8, 100),
            nn.BatchNorm1d(100),
            nn.ReLU(True),
            nn.Linear(100, 2),
        )

    def forward(self, input):
        input = input.view(-1, 256 * 8 * 8)
        return self.net(input)

### Unet

In [11]:
def expand_to_planes(input, shape):
    return input[..., None, None].repeat([1, 1, shape[2], shape[3]])

In [12]:
class Diffusion(nn.Module):
    def __init__(self):
        super().__init__()
        c = 64
        self.feature_embed = FeatureEmbedding()
        self.domain_classifier = DomainClassifier()
        self.timestep_embed = FourierFeatures(1, 16)
        self.class_embed = nn.Embedding(11, 4)

        self.avg_1 = nn.AvgPool2d(2)
        self.avg_2 = nn.AvgPool2d(2)
        self.avg_3 = nn.AvgPool2d(2)

        self.down_0 = nn.Sequential(
            ResConvBlock(3 + 16 + 4, c, c),
            ResConvBlock(c, c, c),
        )

        self.down_1 = nn.Sequential(
            ResConvBlock(c, c * 2, c * 2),
            ResConvBlock(c * 2, c * 2, c * 2),
        )

        self.down_2 = nn.Sequential(
            ResConvBlock(c * 2, c * 4, c * 4),
            SelfAttention2d(c * 4, c * 4 // 64),
            ResConvBlock(c * 4, c * 4, c * 4),
            SelfAttention2d(c * 4, c * 4 // 64),
        )

        self.down_3 = nn.Sequential(
            ResConvBlock(c * 4, c * 8, c * 8),
            SelfAttention2d(c * 8, c * 8 // 64),
            ResConvBlock(c * 8, c * 8, c * 8),
            SelfAttention2d(c * 8, c * 8 // 64),
            ResConvBlock(c * 8, c * 8, c * 8),
            SelfAttention2d(c * 8, c * 8 // 64),
            ResConvBlock(c * 8, c * 8, c * 4),
            SelfAttention2d(c * 4, c * 4 // 64),
        )

        self.up_0 = nn.Upsample(scale_factor=2, mode='bilinear', align_corners=False)

        self.up_1 = nn.Sequential(
            ResConvBlock(c * 8, c * 4, c * 4),
            SelfAttention2d(c * 4, c * 4 // 64),
            ResConvBlock(c * 4, c * 4, c * 2),
            SelfAttention2d(c * 2, c * 2 // 64),
            nn.Upsample(scale_factor=2, mode='bilinear', align_corners=False),
        )

        self.up_2 = nn.Sequential(
            ResConvBlock(c * 4, c * 2, c * 2),
            ResConvBlock(c * 2, c * 2, c),
            nn.Upsample(scale_factor=2, mode='bilinear', align_corners=False),
        )

        self.up_3 = nn.Sequential(
            ResConvBlock(c * 2, c, c),
            ResConvBlock(c, c, 3, is_last=True),
        )

    def forward(self, input, t, cond, type = "Target"):
        timestep_embed = expand_to_planes(self.timestep_embed(t[:, None]), input.shape)
        class_embed = expand_to_planes(self.class_embed(cond + 1), input.shape)

        if (type == "Target"):
            feature_embed = self.feature_embed(input)
            x = torch.cat([feature_embed, timestep_embed, class_embed], dim=1)
        else:
            x = torch.cat([input, timestep_embed, class_embed], dim=1)

        # Down Sample
        down_sample = self.down_0(x)
        identity_0 = nn.Identity()(down_sample)

        down_sample = self.avg_1(self.down_1(down_sample))
        identity_1 = nn.Identity()(down_sample)

        down_sample = self.avg_2(self.down_2(down_sample))
        identity_2 = nn.Identity()(down_sample)

        # Middle Sample
        middle_sample = self.down_3(down_sample)
        middle_sample = self.avg_3(self.up_0(middle_sample))

        reverse_feature = ReverseLayerF.apply(middle_sample, 0.5)
        domain_output = self.domain_classifier(reverse_feature)

        # Up Sample
        up_sample = torch.cat([reverse_feature, identity_2], dim=1)
        up_sample = self.up_1(up_sample)

        up_sample = torch.cat([up_sample, identity_1], dim=1)
        up_sample = self.up_2(up_sample)

        up_sample = torch.cat([up_sample, identity_0], dim=1)

        return self.up_3(up_sample), domain_output

# Recover Image

In [13]:
@torch.no_grad()
def sample(model, x, steps, eta, classes, guidance_scale=1.):
    """
    Draws samples from a model given starting noise.
    """
    ts = x.new_ones([x.shape[0]])

    # Create the noise schedule
    t = torch.linspace(1, 0, steps + 1)[:-1]
    alphas, sigmas = get_alphas_sigmas(t)

    # The sampling loop
    for i in trange(steps):

        # Get the model output (v, the predicted velocity)
        with torch.cuda.amp.autocast():
            x_in = torch.cat([x, x])
            ts_in = torch.cat([ts, ts])
            classes_in = torch.cat([-torch.ones_like(classes), classes])
            v_uncond, v_cond = model(x_in, ts_in * t[i], classes_in).float().chunk(2)
        v = v_uncond + guidance_scale * (v_cond - v_uncond)

        # Predict the noise and the denoised image
        pred = x * alphas[i] - v * sigmas[i]
        eps = x * sigmas[i] + v * alphas[i]

        # If we are not on the last timestep, compute the noisy image for the
        # next timestep.
        if i < steps - 1:
            # If eta > 0, adjust the scaling factor for the predicted noise
            # downward according to the amount of additional noise to add
            ddim_sigma = eta * (sigmas[i + 1]**2 / sigmas[i]**2).sqrt() * \
                (1 - alphas[i]**2 / alphas[i + 1]**2).sqrt()
            adjusted_sigma = (sigmas[i + 1]**2 - ddim_sigma**2).sqrt()

            # Recombine the predicted noise and predicted denoised image in the
            # correct proportions for the next step
            x = pred * alphas[i + 1] + eps * adjusted_sigma

            # Add the correct amount of fresh noise
            if eta:
                x += torch.randn_like(x) * ddim_sigma

    # If we are on the last timestep, output the denoised image
    return pred

# Train Stage

In [14]:
rng = torch.quasirandom.SobolEngine(1, scramble=True)
criterion = nn.CrossEntropyLoss()

In [15]:
def generate_diffussion_target(images, labels):
    t = rng.draw(labels.shape[0])[:, 0].to(device)

    alphas, sigmas = get_alphas_sigmas(t)

    alphas = alphas[:, None, None, None]
    sigmas = sigmas[:, None, None, None]

    noise = torch.randn_like(images)
    noised_reals = images * alphas + noise * sigmas

    targets = noise * alphas - images * sigmas

    return t, noised_reals, targets

def train_model(model, optimizer, source_dl, target_dl, epochs, device):
    target_iter = iter(target_dl)
    src_domain_label = torch.zeros(batch_size).long()
    tgt_domain_label = torch.ones(batch_size).long()

    for epoch in range(epochs):
        model.train()
        for src_images, src_labels in tqdm(source_dl):
            # Source
            src_images, src_labels = src_images.to(device), src_labels.to(device)

            t, noised_src, src_recon_targets = generate_diffussion_target(src_images, src_labels)

            optimizer.zero_grad()

            with torch.cuda.amp.autocast():
                output, domain_output = model(noised_src, t, src_labels, type = "Source")
                loss = F.mse_loss(output, src_recon_targets)


            loss_s_domain = criterion(domain_output, src_domain_label)

            # Target
            tgt_images, tgt_labels = next(target_iter)
            with torch.cuda.amp.autocast():
                output, domain_output = model(tgt_images, t, tgt_labels, type = "Target")

            loss_t_domain = criterion(domain_output, tgt_domain_label)

            # TODO: It should be a class recon classifier here

            loss += (loss_s_domain +  loss_t_domain)

            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()


        print(f"Epoch {epoch+1}: Loss = {loss.item()}")
        noise = torch.randn([10, 3, 32, 32], device=device)
        fakes_classes = torch.arange(10, device=device)
        fakes = sample(model, noise, steps, eta, fakes_classes, guidance_scale)
        fakes = (fakes + 1) / 2
        fakes = torch.clamp(fakes, min=0, max = 1)
        save_image(fakes.data, './output/%03d_train.png' % epoch)

# Training

In [16]:
model = Diffusion().to(device)
optimizer = optim.Adam(model.parameters(), lr=2e-4)
scaler = torch.cuda.amp.GradScaler()

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
/usr/local/lib/python3.10/dist-packages/torch/cuda/amp/grad_scaler.py:126: UserWarning: torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.
  warnings.warn(


In [17]:
train_model(model, optimizer, source_dl, target_dl, epochs=30, device=device)

/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


  0%|          | 0/782 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/torch/amp/autocast_mode.py:250: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn(
/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


KeyboardInterrupt: 

In [ ]:
image = Image.open("./output/009_train.png")
plt.imshow(image)
plt.show()

In [ ]:
image = Image.open("./output/019_train.png")
plt.imshow(image)
plt.show()

In [ ]:
image = Image.open("./output/029_train.png")
plt.imshow(image)
plt.show()

# Train Stage

In [ ]:
'''def train_with_features(model, optimizer, train_dl, feature_dl, epochs, device):
    for epoch in range(epochs):
        model.train()
        for i, ((reals, classes), (features, feature_labels)) in enumerate(zip(train_dl, feature_dl)):
            optimizer.zero_grad()
            reals, classes = reals.to(device), classes.to(device)
            features = features.to(device)

            t = rng.draw(reals.shape[0])[:, 0].to(device)
            alphas, sigmas = get_alphas_sigmas(t)

            alphas = alphas[:, None, None, None]
            sigmas = sigmas[:, None, None, None]

            noise = torch.randn_like(reals)
            noised_reals = reals * alphas + noise * sigmas
            targets = noise * alphas - reals * sigmas

            with torch.cuda.amp.autocast():
                v = model(noised_reals, t, classes, external_features=features)
                loss = F.mse_loss(v, targets)

            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()

        noise = torch.randn([10, 3, 32, 32], device=device)
        fakes_classes = torch.arange(10, device=device)

        fakes = sample(model, noise, steps, eta, fakes_classes, guidance_scale)
        fakes = (fakes + 1) / 2
        fakes = torch.clamp(fakes, min=0, max = 1)
        save_image(fakes.data, './output/%03d_imagine.png' % epoch)'''

In [ ]:
'''# Train the model with features
train_with_features(model, optimizer, train_dl, feature_dl, epochs=40, device=device)'''

In [ ]:
'''def train(epoch):
    for i, (reals, classes) in enumerate(tqdm(train_dl)):
        opt.zero_grad()
        reals = reals.to(device)
        classes = classes.to(device)

        # Draw uniformly distributed continuous timesteps
        t = rng.draw(reals.shape[0])[:, 0].to(device)

        # Calculate the noise schedule parameters for those timesteps
        alphas, sigmas = get_alphas_sigmas(t)

        # Combine the ground truth images and the noise
        alphas = alphas[:, None, None, None]
        sigmas = sigmas[:, None, None, None]
        noise = torch.randn_like(reals)

        # with t increase, alpha goes down and sigmas goes up
        # which means with t increase, noised_reals will have more noise and less image
        noised_reals = reals * alphas + noise * sigmas

        # with t increase, alpha goes down and sigmas goes up
        # which means with t increase, targets will closer to - img
        targets = noise * alphas - reals * sigmas

        # Drop out the class on 20% of the examples
        to_drop = torch.rand(classes.shape, device=classes.device).le(0.2)
        classes_drop = torch.where(to_drop, -torch.ones_like(classes), classes)

        # Compute the model output and the loss.
        with torch.cuda.amp.autocast():
            v = model(noised_reals, t, classes_drop)
            loss = F.mse_loss(v, targets)

        # Do the optimizer step and EMA update
        scaler.scale(loss).backward()
        scaler.step(opt)


        # ema_update(model, model_ema, 0.95 if epoch < 20 else ema_decay)
        scaler.update()

        if i % 50 == 0:
            tqdm.write(f'Epoch: {epoch}, iteration: {i}, loss: {loss.item():g}')

    noise = torch.randn([10, 3, 32, 32], device=device)
    fakes_classes = torch.arange(10, device=device)

    fakes = sample(model, noise, steps, eta, fakes_classes, guidance_scale)
    fakes = (fakes + 1) / 2
    fakes = torch.clamp(fakes, min=0, max = 1)
    save_image(fakes.data, './output/%03d.png' % epoch)
'''

In [ ]:
'''try:
    i = 0
    while True:
        i += 1
        train(i)
except KeyboardInterrupt:
    pass'''